# Hi Wojciech

Hi, Wojchiech. 
Here is my code for cleaning my data - it is kind of a mess and some part is only relevant for the heart rate data I guess. 

I do not have your email, but please write me, if you need help: s144289@student.dtu.dk

Luna

# Data cleaning
Data cleaning by reorganizing the data by user rather than year-month. Duplicates and non-used attributes are removed in the process. 

* [x] Done

In [1]:
import os
import glob
import datetime
import csv
import time
import pandas as pd

In [2]:
#Stuff to Read Avro files
packages = '/home/s144289/spark-avro_2.12-3.0.0.jar'
os.environ["PYSPARK_SUBMIT_ARGS"] = ("--jars {0} pyspark-shell ".format(packages))

# temp folder to store working memory (home path of the notebook does not have a lot of memory)
temp_folder = "/data/work/shared/s144289/temp/"

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, TimestampType, IntegerType, DateType, DoubleType, StructType, StructField, LongType
from pyspark.sql import Window

# Create new context
spark = SparkSession.builder.config("spark.sql.files.ignoreCorruptFiles","true")\
                                        .config("spark.memory.fraction", "0.6")\
                                        .config("spark.local.dir",temp_folder)\
                                        .config("spark.driver.memory",'5g')\
                                        .master("local[15]").getOrCreate()
    
# local[5] how many cores to use (32 in total)
# maybe increase driver memory (.config("spark.driver.memory",'10g'))

## Data reorganizing and cleaning

heart_rate00:
* [x] Drop duplicates

heart_rate
* [x] Drop duplicates
* [x] Exclude phone HR
* [x] Exclude dates > 2019-03 and < 2015 05
* [x] Exclude timestamps_seen < time
* [x] Exclude timestamps_seen - time > +30 days

In [5]:
df_original = spark.read.format("avro").load('/data/src/heartrate/yearmonth=201603/')
df_original.printSchema()

root
 |-- timestamp_seen: long (nullable = true)
 |-- useruuid: string (nullable = true)
 |-- time: string (nullable = true)
 |-- bpm: integer (nullable = true)
 |-- devices: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |    |    |-- id: string (nullable = true)



In [3]:
# path of the data:
heart_rate_path = '/data/src/heartrate/'
path = '/data/work/shared/s144289/heart_rate00'

Tformat = "yyyy-MM-dd'T'HH:mm:ss.SSSZ"
included = [0]*5
total_time = 0
estimated_time = 0
N = len(glob.glob(heart_rate_path+'*'))
i=0
for folder in glob.glob(heart_rate_path+'*'):
    start_time = time.time()
    i+=1
    print(i,'of',N,':',folder[-6:], end = '\r')    
    
    # dataframe with data (does not load until you execute something below)
    df_original = spark.read.format("avro").load(folder)
    included[0]+=df_original.count()

    # Remove duplicates
    df = df_original.dropDuplicates()    
    included[1]+=df.count()
    
#     # Split device column to multiple columns:
#     df = df.withColumn('devices',F.explode('devices')
#                                ).withColumn('devices_name',F.col('devices').getItem('name'))
    
#     # Exclude phone measures *
#     df = df.filter(df.devices_name == F.lit('swr12')).drop('devices')
#     included[2]+=df.count()
    
#     # Remove samples out of daterange:
#     df = df.filter( F.to_date(df['time'],Tformat)<F.lit("2019-04-01") ) 
#     df = df.filter( F.to_date(df['time'],Tformat)>F.lit("2015-04-30") ) #*
#     included[3]+=df.count()
    
#     # Remove timestamp_seen < time and <30 month between
#     # Time in unix time: *
#     df = df.withColumn('time_unix', F.unix_timestamp('time',Tformat) )
#     # timestamp_seen to seconds not ms *
#     df = df.withColumn('timestamp_seen', df.timestamp_seen*0.001)
#     df = df.filter(df.time_unix < df.timestamp_seen)
#     df = df.filter(df.timestamp_seen - df.time_unix < F.lit(30*24*60*60) )
#     included[4]+=df.count()
    
    # Only Keep specific columns:
#     df = df.select('useruuid','time','bpm','devices_name') old! (* = new stuff)
    df = df.select('useruuid','time','bpm') 
    
    # Create user bucket id from the first two characters of the useruuid
    df = df.withColumn('bucket', F.substring('useruuid',0,2))
    
    # Partion by the bucket ID and save as parquet:
    df.write.partitionBy('bucket').mode('append').parquet(path)
    
    # Clear cache
    spark.catalog.clearCache()
            
    end_time = time.time()    
    total_time += (end_time-start_time)    
    estimated_time = (total_time / i) * (N-i)    
    print(i,'of',N,'/',folder[-6:],
          'Total time:',round(total_time,2), 's',
          'Estimated time remaining:',round(estimated_time,2), 's', end = '\r')

print(i,'of',N,'/',folder[-6:],
      'Total time:',round(total_time,2), 's',
      'Estimated time remaining:',round(estimated_time,2), 's    ')
print('Done'," "*1000)
included

47 of 47 / 201903 Total time: 7271.55 s Estimated time remaining: 0.0 s    
Done                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

[2142413238, 2089611309, 0, 0, 0]

In [43]:
included
# 2142413238, 2089611309, 2089611302, 2089563279, 2073946706

[2142413238, 2089611309, 2142413231, 2142364977, 2133110355]

In [7]:
# Total processing time:
total_time/60/60

3.1686478668451312

### Output testing:

In [8]:
pp = '/data/work/shared/s144289/heart_rate/bucket=00/'

In [11]:
dff = spark.read.format("parquet").load(pp)
dff

DataFrame[useruuid: string, time: string, bpm: int]

In [10]:
# dff.show()
dff = dff.withColumn('time',F.to_timestamp(dff.time,"yyyy-MM-dd'T'HH:mm:ss.SSSZ").alias('time'))
dff[dff.useruuid == '001ecb1d-a276-438c-b3d1-b04e38c96881'].orderBy('time').show(10)

+--------------------+-------------------+---+
|            useruuid|               time|bpm|
+--------------------+-------------------+---+
|001ecb1d-a276-438...|2016-04-19 20:27:00| 65|
|001ecb1d-a276-438...|2016-04-19 20:37:00| 72|
|001ecb1d-a276-438...|2016-04-19 20:57:00| 70|
|001ecb1d-a276-438...|2016-04-19 21:07:00| 93|
|001ecb1d-a276-438...|2016-04-19 21:17:00| 72|
|001ecb1d-a276-438...|2016-04-19 21:37:00| 80|
|001ecb1d-a276-438...|2016-04-19 21:47:00| 72|
|001ecb1d-a276-438...|2016-04-19 22:07:00| 68|
|001ecb1d-a276-438...|2016-04-19 22:17:00| 64|
|001ecb1d-a276-438...|2016-04-19 22:27:00| 64|
+--------------------+-------------------+---+
only showing top 10 rows



# The rest in specfic for the heart rate data

## Data cleaning Phase1

* [x] Remove duplicates
* [x] Remove <18 years
* [x] Track sample sizes/reductions

Total time: 19808.41

In [19]:
user_path = '/data/work/shared/safe_data/users/users_demographics_and_country/'
df_user = spark.read.format("parquet").load(user_path)
df_user = df_user.drop('bucket')
print('Number of total user profiles:', df_user.count())

# 2019-03. Trust the folders. Changes in June 2016. Maybe focus on after 2016 June

Number of total user profiles: 6153899


In [20]:
df_user.groupBy('birthyear').count().orderBy('birthyear', ascending = False).show(3)

+---------+-----+
|birthyear|count|
+---------+-----+
|     2017|   16|
|     2016|  125|
|     2015|    4|
+---------+-----+
only showing top 3 rows



In [21]:
# df_user = df_user.where((df_user.birthyear <= 1997) | (df_user.birthyear.isNull()))

# Dataframe of under age users:
df_user = df_user.where((df_user.birthyear > 1997) )
under_age_users = [x[0] for x in df_user.select('useruuid').collect()]
print('Number of total user profiles < 18 years:', df_user.count())

Number of total user profiles < 18 years: 126210


In [22]:
# path of the data:
path = '/data/work/shared/s144289/heart_rate/'
clean_path = '/data/work/shared/s144289/heart_rate_clean_phase1'

Tformat = "yyyy-MM-dd'T'HH:mm:ss.SSSZ"
Included = [0]*4
n_users = [0]*4

total_time = 0
estimated_time = 0
N = len(glob.glob(path+'*/'))
i=0
for folder in glob.glob(path+'*/'):
    start_time = time.time()
    i+=1
    print(i,'of',N,':',folder[-10:], end = '\r')    
    
    df_original = spark.read.format("parquet").load(folder)
    
    Included[0]+=df_original.count()
    n_users[0]+=df_original.agg(F.countDistinct('useruuid')).collect()[0][0]
    print(i,'of',N,':',folder[-10:], '1/4', end = '\r')    
    
    # Remove duplicates
    df = df_original.dropDuplicates()
    
    Included[1]+=df.count()
    n_users[1]+=df.agg(F.countDistinct('useruuid')).collect()[0][0]
    print(i,'of',N,':',folder[-10:], '2/4', end = '\r')
    
    # Remove age <18: (Born before 1997 would be +18 in +2015. Null's are also kept)
#     df = df.join(df_user, on =['useruuid'], how = 'left_semi') old!
#     df = df.filter(~df.useruuid.isin(under_age_users)) # Filter is too slow...
    df = df.join(df_user, on =['useruuid'], how = 'left_anti')
    
    Included[2]+=df.count()
    n_users[2]+=df.agg(F.countDistinct('useruuid')).collect()[0][0]
    print(i,'of',N,':',folder[-10:], '3/4', end = '\r')
    
    # Create user bucket id from the first two characters of the useruuid
    # Partion by the bucket ID and save as parquet:
    df = df.withColumn('bucket', F.substring('useruuid',0,2)
                      ).write.partitionBy('bucket'
                                         ).mode('append').parquet(clean_path)
    print(i,'of',N,':',folder[-10:], '4/4', end = '\r')
    
    # clear GC after each folder (release cast memory)
    spark.catalog.clearCache()
    
            
    end_time = time.time()    
    total_time += (end_time-start_time)    
    estimated_time = (total_time / i) * (N-i)    
    print(i,'of',N,'/',folder[-10:], '5/4',
          'Total time:',round(total_time,2), 's',
          'Estimated time remaining:',round(estimated_time,2), 's', end = '\r')

print(i,'of',N,'/',folder[-10:],
      'Total time:',round(total_time,2), 's',
      'Estimated time remaining:',round(estimated_time,2), 's    ')
print('Done'," "*1000)

Included
n_users

256 of 256 / bucket=9e/ Total time: 7619.2 s Estimated time remaining: 0.0 s    7 sss
Done                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

[116920, 116920, 115744, 0]

In [24]:
Included
# [2073946706, 2041017896, 2035436770, 0]

[2073946706, 2041017896, 2035436770, 0]

In [25]:
n_users
# [116920, 116920, 115744, 0]

[116920, 116920, 115744, 0]

## Data cleaning Phase2

* [x] Remove other specific users: 
    * [x] Use-period less than 7 days in a row (max - min time)
    * [x] less than 770 samples (5x22x7)
* [x] Remove days <20 samples
* [x] Track sample sizes/reductions

In [10]:
folder = "/data/work/shared/s144289/Data_outputs/Users_use_metrics/"
files = glob.glob(folder)
df = spark.read.format("parquet").load(files)
df = df.drop('n_days')
df = df.withColumn('Period',F.col('Period')+1)

df_users_exclude = df[( (df['Period'] < 7) & (df['n_samples'] < 5*22*7) )]

df.count()

115744

In [4]:
df_users_exclude.count()

17401

In [5]:
folder = "/data/work/shared/s144289/Data_outputs/Heart_Rates_per_day_per_user/"

df = spark.read.format("parquet").load(folder)
df = df.drop('mean_bpm')
df.n_bpm = df.n_bpm.cast('int')

df_users_day_exclude = df[df['n_bpm']<20]

In [13]:
df_users_day_exclude.agg(F.countDistinct('Date')).show()

+-----------+
|count(Date)|
+-----------+
|       1411|
+-----------+



In [15]:
df_users_day_exclude.agg(F.countDistinct('useruuid')).show()

+---------------+
|count(useruuid)|
+---------------+
|          88771|
+---------------+



In [14]:
df_users_day_exclude.count()

672107

In [7]:
# path of the data:
path = '/data/work/shared/s144289/heart_rate_clean_phase1/'
clean_path = '/data/work/shared/s144289/heart_rate_clean_phase2'

Tformat = "yyyy-MM-dd'T'HH:mm:ss.SSSZ"
Included = [0]*3
n_users = [0]*3

total_time = 0
estimated_time = 0
N = len(glob.glob(path+'*/'))
i=0
for folder in glob.glob(path+'*/'):
    start_time = time.time()
    i+=1
    print(i,'of',N,':',folder[-10:], end = '\r')    
    
    # dataframe with data (does not load until you execute something below)
    df_original = spark.read.format("parquet").load(folder)
    
    Included[0]+=df_original.count()
    n_users[0]+=df_original.agg(F.countDistinct('useruuid')).collect()[0][0]
    print(i,'of',N,':',folder[-10:], '1/4', end = '\r')    

    
    # Exclude users with less than 7 continous days and 770 samples
    df = df_original.join(df_users_exclude[df_users_exclude['bucket'] == folder[-3:-1] ]
                          , on ='useruuid', how = 'left_anti')
    
    Included[1]+=df.count()
    n_users[1]+=df.agg(F.countDistinct('useruuid')).collect()[0][0]
    print(i,'of',N,':',folder[-10:], '2/4', end = '\r')   
    
    
    # Exclude user-days with less than 20 samples:
    df = df.withColumn('Date',F.to_date('time',Tformat))    
    df = df.join(df_users_day_exclude[df_users_day_exclude['bucket'] == folder[-3:-1] ]
                 , on =['useruuid','Date'], how = 'left_anti')
    
    Included[2]+=df.count()
    n_users[2]+=df.agg(F.countDistinct('useruuid')).collect()[0][0]
    print(i,'of',N,':',folder[-10:], '3/4', end = '\r')   
    
    
    # Create user bucket id from the first two characters of the useruuid
    # Partion by the bucket ID and save as parquet:
    df = df.withColumn('bucket', F.substring('useruuid',0,2)
                      ).write.partitionBy('bucket'
                                         ).mode('append').parquet(clean_path)
    print(i,'of',N,':',folder[-10:], '4/4', end = '\r')
    
    # clear GC after each folder (release cast memory)
    spark.catalog.clearCache()
    
            
    end_time = time.time()    
    total_time += (end_time-start_time)    
    estimated_time = (total_time / i) * (N-i)    
    print(i,'of',N,'/',folder[-10:], '5/4',
          'Total time:',round(total_time,2), 's',
          'Estimated time remaining:',round(estimated_time,2), 's', end = '\r')

print(i,'of',N,'/',folder[-10:],
      'Total time:',round(total_time,2), 's',
      'Estimated time remaining:',round(estimated_time,2), 's    ')
print('Done'," "*1000)

print(Included)
print(n_users)

256 of 256 / bucket=9e/ Total time: 13067.45 s Estimated time remaining: 0.0 s    s ss
Done                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [52]:
# [2035436770, 2032886932, 2028063167]
# [115744, 100409, 99733]

### Pre phase 2 limit analysis (Users us metrics):
* Number of samples
* Number of days
* Periode (max date to min date)

In [38]:
# path of the data:
path = '/data/work/shared/s144289/heart_rate_clean_phase1/'
clean_path = '/data/work/shared/s144289/Data_outputs/Users_use_metrics'

Tformat = "yyyy-MM-dd'T'HH:mm:ss.SSSZ"

total_time = 0
estimated_time = 0
N = len(glob.glob(path+'*/'))
i=0
for folder in glob.glob(path+'*/'):
    start_time = time.time()
    i+=1
    print(i,'of',N,':',folder[-10:], end = '\r')    
    
    # dataframe with data (does not load until you execute something below)
    df_original = spark.read.format("parquet").load(folder)
      
    df = df_original.groupby('useruuid').agg(F.count('bpm').alias('n_samples'),
                                             F.countDistinct(F.to_date('time')).alias('n_days'),
                                             F.datediff(F.max('time'),F.min('time')).alias('Period')
                                            )
    
    
    # Create user bucket id from the first two characters of the useruuid
    # Partion by the bucket ID and save as parquet:
    df = df.withColumn('bucket', F.substring('useruuid',0,2)
                      ).write.partitionBy('bucket'
                                         ).mode('append').parquet(clean_path)
    print(i,'of',N,':',folder[-10:], '4/4', end = '\r')
    
    # clear GC after each folder (release cast memory)
    spark.catalog.clearCache()
            
    end_time = time.time()    
    total_time += (end_time-start_time)    
    estimated_time = (total_time / i) * (N-i)    
    print(i,'of',N,'/',folder[-10:], '5/4',
          'Total time:',round(total_time,2), 's',
          'Estimated time remaining:',round(estimated_time,2), 's', end = '\r')

print(i,'of',N,'/',folder[-10:],
      'Total time:',round(total_time,2), 's',
      'Estimated time remaining:',round(estimated_time,2), 's    ')
print('Done'," "*1000)


256 of 256 / bucket=9e/ Total time: 3813.62 s Estimated time remaining: 0.0 s    ssss
Done                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              